# Test of the output of the new RNA-seq analysis snakemake workflow

## HTseq output

The workflow produces a count table obtained with HTSEQ named `htseq_genes.txt` (tab separated table). You can find the count table in the folder: `results/htseq/genes`

It should look like this: 

`less htseq_genes.txt` 

```
ENSMUST00000000001.4    3796    2340    2380    1157    2312    2075    2430    2144    2311    2454    2255
ENSMUST00000000003.13   0       0       0       0       0       0       0       0       0       0       0
ENSMUST00000000010.8    1       0       0       0       0       0       0       3       3       0       0
ENSMUST00000000028.13   0       0       0       0       0       0       0       0       0       0       0
ENSMUST00000000033.11   0       0       0       0       0       0       0       0       0       0       0
ENSMUST00000000049.5    0       0       0       0       1       0       0       0       0       0       0
```

### Setup the Docker image to perform the analysis on your own computer

To perform the analysis on your own computer, you will need the administartors rights. 

- Install Docker on your computer by downloading it the version corresponsing to your [OS](https://docs.docker.com/get-docker/). Follow the instructions for the installation. 

- Once the installation is done, start `Docker Desktop`

- Prepare a folder (refered from now as `working directory` where you will perform the analysis. The analysis requires 2 files:
    - The count table `htseq_genes.txt`.
    - The `units.tsv` you used for the analysis.

- Then using your usual Terminal (I recommand [Visual code studio](https://code.visualstudio.com/download)), open a terminal. 

- Change directory in order to get to the `working directory`

- Set up the variable `WD` with the following command:

```
WD=pwd
``` 

- Check that the variable is set correctly with the following command: `$WD`. This command should return the absolute path of the `working directory`. 

- Initiate Docker with: 

```
docker run --rm -it -d -v $WD  jchouaref/bioconductor_biomart:1.0
```

If it initate correclty then, the name of the image container should appear in the terminal: `ab79cc2dfb22f01b9198706bf46c2faa02c6814a5dfcd4d570844a963ef3b889`

- You should be then able to start a R session by typing: `R` 

### Perform the analysis in R 

Once the R session is initated, start by initiating the variable we will use for the analysis with the following:
```
count_df <- "featureCounts_genes.txt"
file_experiment <- "units.tsv"
```

Write here either the absolute or relative path to the output folder where the output of DESeq2 will be saved: 

```
output <- "results/"
```

Load the required R packages:

```
library(ggplot2)
library(DESeq2)
library("RColorBrewer")
library(pheatmap)
library("biomaRt")
library(Glimma)
library(EnhancedVolcano)
library(readxl)
library(dplyr)
library(readr)
```

### Prepare the count table using the `units.tsv` file 

The units.tsv file contains the files names and the conditions, you will need to use for the analysis.

- First set up the condition variables:

```
condition1 <- "treatment"
condition2 <- "control"
```

The conditions need to have the same name as what you defined in the `units.tsv` file.

- Load the `units.tsv` table and save it in the variable: `design_exp`: `design_exp <- read.table(file_experiment, header = TRUE, row.names = "sample")`

- Then load the count table and set the header names :
    - First load the count table with the following:
    ```
    count_table <- read_delim(count_df,
    delim = "\t",  escape_double = FALSE,
    trim_ws = TRUE)
    ```

    The count table might not have headers. Headers are necessary for the script.
    They also need to have the same name as the samples for the `design_exp` table
    You can set names for the headers for this file.

    - Set the header to the `count_table` with the following:
    ``` 
    colnames(count_table) <- c("Gene", "RNA1", "RNA2", "RNA3", "RNA4", "RNA5", "RNA6")
    ```
The column headers are in the **exact same order** as in the `units.tsv` file. If there is a typo in the names or a change in their order, R will raise a error message.

- Prepare count table to perform the DESeq2 aanalysis with the following commands:

```
row.names(count_table) <- count_table$Gene

count_table_df <- count_table[, 2:ncol(count_table)]

row.names(count_table_df) <- count_table$Gene

design_exp <- design_exp[names(count_table_df), ]

design_exp$condition <- factor(design_exp$condition)

```

### Running the DESeq2 analysis

```
dds <- DESeqDataSetFromMatrix(countData = count_table_df,
                            colData = design_exp,
                            design = ~condition)

dds$condition <- relevel(dds$condition, ref = "control")

dds <- DESeq(dds)

res2 <- results(dds, contrast = c("condition", condition1, condition2 ))

rld = rlog(dds, blind = TRUE) # nolint

```

#### Save output table

```
write.table(res2,
    file = paste0(output, "deseq2_table_", condition1, "_vs_", condition2, ".txt"),
    sep = "\t",
    quote = FALSE)
```

This table can be read using Excel and already contain all the information needed to know if there are differentially expressed genes. However the gene names are ensembl gene ID. In the next steps, we will transform this table to add the gene symbols to it as well as the read count and normalized read count per gene for each sample.

#### PDF with QC measurments: PCA, correlation heatmap

```
pdf(file = paste0(output, "MAplot_DGE_", condition1, "_vs_", condition2, ".pdf"),
    paper = "a4")
plotMA(res2)
rld <- rlog(dds, blind = TRUE)
plotPCA(rld,
    n = 10000)

sample_dists <- dist(t(assay(rld)))
sample_dist_matrix <- as.matrix(sample_dists)
rownames(sample_dist_matrix) <- rld$condition
colnames(sample_dist_matrix) <- NULL
colors <- colorRampPalette(rev(brewer.pal(9, "Blues")))(255)

pheatmap(sample_dist_matrix,
    clustering_distance_rows = sample_dists,
    clustering_distance_cols = sample_dists,
    col = colors)
dev.off()
```

#### Biomart annotation

```
genes   <- as.character(row.names(count_table_df))
fixed_names = sapply(strsplit(genes, ".", fixed=T), function(x) x[1])

mart    <- useDataset("mmusculus_gene_ensembl", useMart("ensembl"))

G_list  <- getBM(filters = "ensembl_gene_id",
            attributes = c("ensembl_gene_id",'mgi_symbol', 'chromosome_name',
            'start_position', 'end_position', 'strand',
            "description"),
            values = fixed_names,
            mart = mart)

annotation <- aggregate(G_list[, "mgi_symbol"],
    by = list(G_list$ensembl_gene_id, G_list$chromosome_name,
            G_list$start_position, G_list$end_position,
            G_list$strand,
            G_list$description),
            function(x){paste(x, sep="_")})

annotation <- aggregate(mgi_symbol~ensembl_gene_id+chromosome_name+start_position+end_position+strand+description,
    data=G_list,
    FUN = paste,
    collapse = ",")

row.names(annotation) <- annotation$ensembl_gene_id

annotation$name <- unlist(annotation$mgi_symbol)
```

#### Glimma

```
genes   <- as.character(row.names(res2))
fixed_names = sapply(strsplit(genes, ".", fixed=T), function(x) x[1])

annotation <- annotation[fixed_names, ]

annotation$Gene <- annotation$ensembl_gene_id

dt.res2 <- as.numeric(res2$padj < 0.05)

row.names(res2) <- fixed_names

finalTable <- merge(annotation,
    as.data.frame(res2),
    by.x = "row.names",
    by.y = "row.names")

row.names(finalTable) <- finalTable$Row.names

finalTable$Row.names <- c()

dds_count <- counts(dds, normalized = FALSE)
genes   <- as.character(row.names(dds_count))
fixed_names = sapply(strsplit(genes, ".", fixed=T), function(x) x[1])
row.names(dds_count) <- fixed_names

finalTable <- merge(finalTable,
    dds_count,
    by.x = "row.names",
    by.y = "row.names")

row.names(finalTable) <- finalTable$Row.names

finalTable$Row.names <- c()

dds_count_norm <- counts(dds, normalized = TRUE)
genes   <- as.character(row.names(dds_count_norm))
fixed_names = sapply(strsplit(genes, ".", fixed=T), function(x) x[1])
row.names(dds_count_norm) <- fixed_names

finalTableNorm <- merge(finalTable,
    dds_count_norm,
    by.x = "row.names",
    by.y = "row.names",
    suffixes = c("", ".norm"))

``` 
The following command will open the browser to display a MDS plot. It's a plot that discplay the **Euclidian distances** between your samples. It helps to know if the biological replicates are close to one another and also have a first idea of how similar the 2 conditions tested are.

```
glMDSPlot(dds, groups = design_exp, launch = TRUE)
```

The following command will also open the browser to display a MA plot, the MA plot will the log fold change measured for every genes. The genes above 0 are more expressed in the `treatment` condition compared to the `control` condition , while the genes below the 0 aree less expressed in the `treatment` condition compared to the `control` condition.

```
glMDPlot(res2, status = dt.res2,
    counts = counts(dds, normalized = TRUE),
    groups = dds$condition,
    transform = TRUE,
    side.main = "mgi_symbol",
    samples = colnames(dds),
    anno = annotation[, c(9, 7, 6)],
    launch = TRUE)
```

### Save the final table 

write.table(finalTableNorm,
    file = paste0(output, "DESEQ2_final_table_", condition1, "_vs_", condition2, ".txt"), 
    sep = "\t", 
    quote = FALSE, 
    row.names = FALSE)


This is the table, you will want to use in excel to make the filtering for the padj and the log2FC. Since it contains the information about the counts and normalized counts per genes, it can also be useful to make heatmaps and GSEA.

### Enhanced volcano plot

The following part will show how to prepare a volcano plot in R. Note that the final table can also be loaded directly in [VolcanoseR](https://huygens.science.uva.nl/VolcaNoseR/) to produce volcano plot very easily.

```
res3 <- finalTableNorm %>%
  select(name, baseMean, log2FoldChange, lfcSE, stat, pvalue, padj)
```

The values below will need to be adapted to each volcanoplot:

```
min_x_lim <- -4
max_x_lim <- 5
min_y_lim <- 0
max_y_lim <- 20
title_plot <- "Volcano plot DGE treatment vs control"
```

```
pdf(paste0(output, "Enhanced_volcano_plot_", condition1, "_vs_", condition2, ".pdf"), height = 10, width = 10)
  EnhancedVolcano(res3,
    lab = res3$name,
    subtitle = "",
    x = "log2FoldChange",
    y = "padj",
    xlim = c(min_x_lim, max_x_lim),
    ylim = c(min_y_lim, max_y_lim),
    pCutoff = 0.05,
    ylab = "-Log10 padj value",
    title = title_plot,
    )
  
dev.off()
```

